In [28]:
athena_db = 'trivago'
s3_bucket = 'athena-trivago-auto'
date = '2019-08-01'

In [14]:
import awswrangler
session = awswrangler.Session()

In [15]:
import pandas

# Get USPs per accommodation and add destination_id

In [16]:
def get_destination_ids(accommodation_ids):
    with open('query.sql') as f:
        df = session.pandas.read_sql_athena(
            sql=f.read().format(date=repr(date)),
            database=athena_db
        )
        
    return df[df.accommodation_id.isin(accommodation_ids)]

In [17]:
def get_filtered_usps():
    df = session.pandas.read_sql_athena(
        sql="SELECT * FROM unique_selling_points WHERE date = {date}".format(date=repr(date)),
        database=athena_db
    )
    
    return df

In [25]:
usp_df = get_filtered_usps()
destination_ids = get_destination_ids(usp_df["accommodation_id"].values)
usp_df = usp_df.merge(destination_ids, how="left", on="accommodation_id")

In [26]:
usp_df.sample(5)

,accommodation_id,usp,accommodation_ns,date,destination_id
357735,457736,Exquisite Experience,100,2019-08-01,NaN
402216,502217,Downtown,100,2019-08-01,NaN
11943,22088,hot_tub,100,2019-08-01,15246
106474,206475,Luxury hotel,100,2019-08-01,NaN
136947,10171,Business hotel,100,2019-08-01,10171


In [27]:
usp_df.groupby("usp")['destination_id'].nunique().sort_values(ascending=False)

usp
Business hotel        2300
sea_view              2275
Sleeping comfort      2270
Pet friendly hotel    2241
hot_tub               1869
                      ... 
sunday_brunch           10
live_music               9
skyline_view             8
japanese_style           7
blackout_curtains        3
Name: destination_id, Length: 63, dtype: int64

In [29]:
session.pandas.to_parquet(
    dataframe=usp_df,
    path=f"s3://{s3_bucket}/usp/output",
    partition_cols=["date"],
)

['s3://athena-trivago-auto/usp/output/date=2019-08-01/f3844dc1f0974712974031c6179763be.parquet',
 's3://athena-trivago-auto/usp/output/date=2019-08-01/72ea1c1490f64ee7a0e2304be62decae.parquet']